## Part 1: Preprocessing

In [174]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [175]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [176]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [177]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
x_df = attrition_df[X_columns]

# Show the data types for X_df
x_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [178]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, random_state=78)

In [179]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['Overtime_Binary'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['Overtime_Binary'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

X_train = X_train.drop('OverTime', axis=1)
X_test = X_test.drop('OverTime', axis=1)
X_train['Overtime_Binary'].value_counts()


Overtime_Binary
0    799
1    303
Name: count, dtype: int64

In [180]:
# Create a StandardScaler
standard_scaler = StandardScaler()

# Fit the StandardScaler to the training data
standard_scaler.fit(X_train)

# Scale the training and testing data
X_test_scaled = standard_scaler.transform(X_test)
X_train_scaled = standard_scaler.transform(X_train)




In [181]:
# Create a OneHotEncoder for the Department column
dep_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
dep_encoder.set_output(transform="pandas")
# Fit the encoder to the training data
dep_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
encoded_dept_train = dep_encoder.transform(y_train[['Department']])
encoded_dept_test = dep_encoder.transform(y_test[['Department']])

y_train = pd.concat([y_train, encoded_dept_train], axis=1)
y_test = pd.concat([y_test, encoded_dept_test], axis=1)

y_train = y_train.drop('Department', axis=1)
y_test = y_test.drop('Department', axis=1)



In [ ]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
attr_encoder.set_output(transform="pandas")
# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
encoded_attr_train = attr_encoder.transform(y_train[['Attrition']])
encoded_attr_test = attr_encoder.transform(y_test[['Attrition']])

y_train = pd.concat([y_train, encoded_attr_train], axis=1)
y_test = pd.concat([y_test, encoded_attr_test], axis=1)

y_train = y_train.drop('Attrition', axis=1)
y_test = y_test.drop('Attrition', axis=1)



,Department_Human Resources,Department_Research & Development,Department_Sales,Attrition_No,Attrition_Yes
591,0.0,0.0,1.0,0.0,1.0
267,0.0,1.0,0.0,1.0,0.0
1236,0.0,0.0,1.0,0.0,1.0
788,0.0,1.0,0.0,1.0,0.0
1224,0.0,1.0,0.0,1.0,0.0


## Create, Compile, and Train the Model

In [183]:
# Find the number of columns in the X training data


# Create the input layer


# Create at least two shared layers


In [184]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [185]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [186]:
# Create the model


# Compile the model


# Summarize the model


In [187]:
# Train the model



In [188]:
# Evaluate the model with the testing data


In [189]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 